In [1]:
# Initial Imports
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [2]:
# Load and verify the Enviornment Factors
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

In [3]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
print(type(alpaca_api_key))
print(type(alpaca_secret_key))

<class 'NoneType'>
<class 'NoneType'>


In [4]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version= "v2")

ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

In [27]:
today = pd.Timestamp('2022-12-22', tz="America/New_York").isoformat()

In [28]:
timeframe = "1Day"

In [29]:
df_portfolio =alpaca.getbars(
    tickers,
    timerframe,
    start = '2020-01-01',
    end = today
).df

NameError: name 'alpaca' is not defined